In [7]:
    # from selenium import webdriver
# import time
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.support.ui import Select
# from selenium.webdriver.common.by import By
# import os

In [8]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup
import os
import pandas as pd

In [9]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [10]:
seasons = list(range(2021,2022))
seasons

[2021]

In [11]:
# Set up the browser
executable_path = {'executable_path':"C:\Program Files (x86)\msedgedriver.exe"}
browser = Browser('edge', **executable_path)

In [12]:
for season in seasons:
    # Visit the website for scraping
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    browser.visit(url)

    # Create a BeautifulSoup object
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    filter_div = soup.find('div', class_='filter')
    links = filter_div.find_all('a')
    urls = [link.get("href") for link in links]

In [13]:
table = soup.find('table', id='schedule')

links = table.find_all("a")
hrefs = [link.get('href') for link in links]
box_scores = [link for link in hrefs if link and "boxscore" in link and ".html" in link]
box_scores = [f"https://www.basketball-reference.com{score}" for score in box_scores]

In [14]:
box_score = box_scores[0]

browser.visit(box_score)

html=browser.html
soup = BeautifulSoup(html, 'html.parser')

In [15]:
score_table = soup.find('table', id='line_score')
tbody = score_table.find('tbody')
rows = tbody.find_all('tr')

line_score = []

for row in rows:
    # Get team name
    team = row.find('th', class_='center').text
    columns = row.find_all('td')
    
    # Create line score dictionary
    if(columns !=[]):
        total = columns[4].text
    
    # Create dictionary for dataframe later
    line_score_dict = { "team": team,
                        "total": total
    }
    
    # Add dictionary to array
    line_score.append(line_score_dict)

In [16]:
# Create Data frame
df = pd.DataFrame(line_score)
df

,team,total
0,GSW,99
1,BRK,125


In [17]:
# Get basic and advanced box scores
teams = []
for i in range(len(line_score)):
    teams.append(line_score[i]['team'])
teams

['GSW', 'BRK']

In [27]:
stats = ['basic', 'advanced']
home_team_tables = {'basic': [], 'advanced': []}
away_team_tables = {'basic': [], 'advanced': []}

for team in teams:
    for stat in stats:
        stat_table = soup.find('table', id=f"box-{team}-game-{stat}")
        table_data = []
        if stat_table:
            for row in stat_table.find_all('tr'):
                row_data = [cell.get_text() for cell in row.find_all(['th', 'td'])]
                table_data.append(row_data)
                
        # Store table data on the type of stat team
        if team == teams[0]:
            home_team_tables[stat].extend(table_data)
        elif team == teams[1]:
            away_team_tables[stat].extend(table_data)

In [146]:
home_advanced = pd.DataFrame(home_team_tables['advanced']).dropna().drop(index=7)
home_basic = pd.DataFrame(home_team_tables['basic']).dropna().drop(index=7)
away_advanced = pd.DataFrame(away_team_tables['advanced']).dropna().drop(index=7)
away_basic = pd.DataFrame(away_team_tables['basic']).dropna().drop(index=7)

In [147]:
# Use first row as headers
advanced_headers = home_advanced.iloc[0]
basic_headers = home_basic.iloc[0]


# Set headers
home_advanced.rename(columns=advanced_headers, inplace=True)
away_advanced.rename(columns=advanced_headers, inplace=True)

home_basic.rename(columns=basic_headers, inplace=True)
away_basic.rename(columns=basic_headers, inplace=True)

home_advanced = home_advanced.drop(index=1).reset_index(drop=True)
away_advanced = away_advanced.drop(index=1).reset_index(drop=True)
home_basic = home_basic.drop(index=1).reset_index(drop=True)
home_advanced = home_advanced.drop(index=1).reset_index(drop=True)

In [151]:
away_basic.head()

,Starters,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
1,Starters,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
2,Kyrie Irving,25:18,10,16,.625,4,7,.571,2,2,...,1,3,4,4,0,0,1,3,26,+32
3,Kevin Durant,24:56,7,16,.438,1,2,.500,7,7,...,1,4,5,3,3,1,1,3,22,+26
4,Joe Harris,20:46,4,8,.500,2,5,.400,0,0,...,2,5,7,2,0,1,2,2,10,+21
5,Spencer Dinwiddie,19:57,2,6,.333,1,3,.333,4,4,...,0,5,5,4,0,0,3,1,9,+24
